<a href="https://colab.research.google.com/github/BTejas001/MWPS_V1/blob/main/mwps_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import pandas as pd
from torch.utils.data import Dataset

# Step 1: Load the dataset
df = pd.read_csv("/content/upmat.csv")  # Ensure this path is correct

# Step 2: Prepare the Dataset class for GPT-2
class MathDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=50):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        input_text = self.dataframe.iloc[idx]["input"]
        target_text = self.dataframe.iloc[idx]["target"]

        # Combine input and target for GPT-2
        text = f"Problem: {input_text}\nEquation: {target_text}"
        encoding = self.tokenizer(text, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": encoding["input_ids"].flatten()  # For causal language modeling, labels are the same as input_ids
        }

# Initialize and configure the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add padding token if not already present
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids('[PAD]')

# Instantiate the dataset with the loaded data
dataset = MathDataset(df, tokenizer, max_length=50)

# Step 4: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
)

# Step 5: Initialize Trainer
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Resize embeddings to include new padding token

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Step 6: Train the model
trainer.train()

# Step 7: Inference - Generate an equation from a new word problem
def generate_equation(word_problem):
    input_text = f"Problem: {word_problem}\nEquation:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate output using the model
    outputs = model.generate(input_ids, max_length=50, num_return_sequences=1, pad_token_id=tokenizer.pad_token_id)

    # Decode the output
    equation = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the equation part
    equation = equation.split("Equation:")[-1].strip()

    return equation

# Example usage
word_problem = "Tom has 15 apples and includes 3 more from his friend."
equation = generate_equation(word_problem)
print(equation)  # Expected: "15 + 3"


Step,Training Loss
10,81.967600
20,78.191900
30,67.311100
40,58.728600
50,38.692100
60,23.586900
70,10.521300
80,6.602700
90,5.102300
100,4.293100


(15.0+3.0)


In [5]:
word_problem = "Tom has 1 apples and buys 3 more from his friend."
equation = generate_equation(word_problem)
print(equation)

1.0*3.0
 is 1.0*(1.0-3.0)


In [7]:
word_problem = "It takes 4 apples to make 1 pie . How many apples does it take to make 504 pies ?"
equation = generate_equation(word_problem)
print(equation)

(4.0*(4.0-1.0))
Equation


In [8]:
word_problem = "JOhn has 5 apples. He adds 3 more apples"
equation = generate_equation(word_problem)
print(equation)

(5.0+3.0)


In [11]:
word_problem = "JOhn has 5 apples. He subtracts 3 apples"
equation = generate_equation(word_problem)
print(equation)

(


In [12]:
word_problem = "Dam has 5 apples. He adds 3 more apples"
equation = generate_equation(word_problem)
print(equation)

5.0/(3.0-3.0)


In [14]:
word_problem = "man has 5 apples. He adds 6 more apples"
equation = generate_equation(word_problem)
print(equation)

(


In [16]:
## To save the model to local storage

model_save_path = 'C:\\Users\\ROG\\Desktop\\mat\\gptt2\\model_dir'
tokenizer_save_path = 'C:\\Users\\ROG\\Desktop\\mat\\gptt2\\tokenizer_dir'
# Create directories if they don't exist
import os
os.makedirs(model_save_path, exist_ok=True)
os.makedirs(tokenizer_save_path, exist_ok=True)

# Save model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)


('C:\\Users\\ROG\\Desktop\\mat\\gptt2\\tokenizer_dir/tokenizer_config.json',
 'C:\\Users\\ROG\\Desktop\\mat\\gptt2\\tokenizer_dir/special_tokens_map.json',
 'C:\\Users\\ROG\\Desktop\\mat\\gptt2\\tokenizer_dir/vocab.json',
 'C:\\Users\\ROG\\Desktop\\mat\\gptt2\\tokenizer_dir/merges.txt',
 'C:\\Users\\ROG\\Desktop\\mat\\gptt2\\tokenizer_dir/added_tokens.json')

In [ ]:
## To load the model from local storage

from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_save_path = '/content/model_directory'
tokenizer_save_path = '/content/tokenizer_directory'

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained(model_save_path)
tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_save_path)
